In [1]:
import pandas as pd
import json
import csv

In [2]:
musicdf = pd.read_csv('Cleaned.csv') 
musicdf

,Song,Song genres,Artist(s),Spotify stream Total,Year Released
0,The Night We Met,"Alternative, Rock",Lord Huron,2715009662,2015
1,Love Yourself,Pop,Justin Bieber,2653556700,2015
2,The Hills (clean),"Electro, Rock, R&B",The Weeknd,2539779204,2015
3,Stressed Out,"Alternative, Electro, Pop",twenty one pilots,2539737195,2015
4,The Hills,"R&B, Electro, Rock",The Weeknd,2538736910,2015
...,...,...,...,...,...
20756,Twistin' The Night Away - Restored Edition '25,Pop,Sam Cooke,101101751,2025
20757,Dance With Me/ Sing To You - Original Mix,Hip Hop,"SAICHI SU, Shiloh Dynasty",98760143,2025
20758,VeLDÁ,Latin,"Bad Bunny, Dei V, Omar Courtz",92309602,2025
20759,EoO,Latin,Bad Bunny,84646945,2025


In [3]:
musicdf.dtypes


Song                    object
Song genres             object
Artist(s)               object
Spotify stream Total     int64
Year Released            int64
dtype: object

In [4]:
# Create empty arrays for each year's data
tracks_by_year = {}
streams_by_year = {}
 
# Loop through the years 2015-2024
for year in range(2015, 2025):
    # Filter data for each year
    result = musicdf.loc[musicdf['Year Released'] == year]
    year_df = pd.DataFrame(result)
 
    # Get total streams for each genre in that year
    genre_streams = {}
 
    # Loop through each row to process genres
    for index, row in year_df.iterrows():
        genres = str(row['Song genres']).split(',')  # Split multiple genres
        for genre in genres:
            genre = genre.strip().lower()  # Clean up genre name
            if genre:  # If genre is not empty
                # Add streams to genre total
                if genre in genre_streams:
                    genre_streams[genre] = genre_streams[genre] + row['Spotify stream Total']
                else:
                    genre_streams[genre] = row['Spotify stream Total']
 
    # Store results for this year
    tracks_by_year[year] = len(year_df)
    streams_by_year[year] = genre_streams
 
# Create a DataFrame with the results
genres_over_time = pd.DataFrame()
 
# Get unique genres across all years
all_genres = set()
for year_data in streams_by_year.values():
    all_genres.update(year_data.keys())
 
# Create rows for each year and genre
data_rows = []
for year in range(2015, 2025):
    for genre in all_genres:
        streams = streams_by_year[year].get(genre, 0)
        data_rows.append({
            'Year': year,
            'Genre': genre,
            'Total Streams': streams
        })
 
# Convert to DataFrame
genres_df = pd.DataFrame(data_rows)
 
# Display first few rows
print(genres_df)
# Optional: Save to CSV
genres_df.to_csv('genre_streams_by_year.csv', index=False)

     Year         Genre  Total Streams
0    2015         metal     6742459181
1    2015       electro   142192759681
2    2015         latin    35015426318
3    2015          rock   221928660609
4    2015  instrumental      263915373
..    ...           ...            ...
255  2024        spoken              0
256  2024    spirituals      171902560
257  2024          jazz     7691115696
258  2024          mena      302425238
259  2024     classical     1139616969

[260 rows x 3 columns]


In [5]:
track2015array = []
stream2015array = []

In [6]:
# Import required libraries
import json
import pandas as pd
 
# Create empty arrays for each year's data
tracks_by_year = {}
streams_by_year = {}
 
# Loop through the years 2015-2024
for year in range(2015, 2025):
    # Filter data for each year
    result = musicdf.loc[musicdf['Year Released'] == year]
    year_df = pd.DataFrame(result)
 
    # Get total streams for each genre in that year
    genre_streams = {}
 
    # Loop through each row to process genres
    for index, row in year_df.iterrows():
        genres = str(row['Song genres']).split(',')
        for genre in genres:
            genre = genre.strip().lower()
            if genre:
                if genre in genre_streams:
                    genre_streams[genre] = genre_streams[genre] + row['Spotify stream Total']
                else:
                    genre_streams[genre] = row['Spotify stream Total']
 
    # Store results for this year
    tracks_by_year[year] = len(year_df)
    streams_by_year[year] = genre_streams
 
# Create a DataFrame with the results
all_genres = set()
for year_data in streams_by_year.values():
    all_genres.update(year_data.keys())
 
# Create rows for each year and genre
data_rows = []
for year in range(2015, 2025):
    for genre in all_genres:
        streams = streams_by_year[year].get(genre, 0)
        data_rows.append({
            'Year': year,
            'Genre': genre,
            'Total Streams': streams
        })
 
# Convert to DataFrame
genres_df = pd.DataFrame(data_rows)
 
# Create combined JSON data structure
combined_json_data = {
    'raw_data': [],
    'genre_analysis': {
        'yearly_stats': {}
    }
}
 
# Add raw track data
for index, row in musicdf.iterrows():
    track_data = {
        'song': row['Song'],
        'song_genres': row['Song genres'],
        'artist': row['Artist(s)'],
        'spotify_streams': int(row['Spotify stream Total']),
        'year_released': int(row['Year Released']),
        'index': index
    }
    combined_json_data['raw_data'].append(track_data)
 
# Add genre analysis data
for year in range(2015, 2025):
    year_data = genres_df[genres_df['Year'] == year]
    combined_json_data['genre_analysis']['yearly_stats'][str(year)] = {
        'genres': {}
    }
 
    for _, genre_row in year_data.iterrows():
        combined_json_data['genre_analysis']['yearly_stats'][str(year)]['genres'][genre_row['Genre']] = {
            'total_streams': int(genre_row['Total Streams'])
        }
 
# Save to JSON file
with open('spotify_complete_analysis.json', 'w', encoding='utf-8') as f:
    json.dump(combined_json_data, f, indent=4, ensure_ascii=False)
 
# Print confirmations and preview
print("Complete JSON file has been created!")
print("\nFirst few records of raw data:")
print(json.dumps(combined_json_data['raw_data'][:2], indent=4))
print("\nFirst year of genre analysis:")
print(json.dumps(dict(list(combined_json_data['genre_analysis']['yearly_stats'].items())[:1]), indent=4))

Complete JSON file has been created!

First few records of raw data:
[
    {
        "song": "The Night We Met",
        "song_genres": "Alternative, Rock",
        "artist": "Lord Huron",
        "spotify_streams": 2715009662,
        "year_released": 2015,
        "index": 0
    },
    {
        "song": "Love Yourself",
        "song_genres": "Pop",
        "artist": "Justin Bieber",
        "spotify_streams": 2653556700,
        "year_released": 2015,
        "index": 1
    }
]

First year of genre analysis:
{
    "2015": {
        "genres": {
            "metal": {
                "total_streams": 6742459181
            },
            "electro": {
                "total_streams": 142192759681
            },
            "latin": {
                "total_streams": 35015426318
            },
            "rock": {
                "total_streams": 221928660609
            },
            "instrumental": {
                "total_streams": 263915373
            },
            "sports": {
 